In [13]:
# Import necessary library

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import csv
import requests

In [7]:
# Produce a url from position and location

def get_url(position,location):
    template="https://uk.indeed.com/jobs?q={}&l={}"
    url=template.format(position,location)
    return url

In [15]:
# Check if the function works

url=get_url("data scientist","London")
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'}
r = requests.get(url,headers=headers)
r.status_code

200

# Extract raw html

In [18]:
# Create driver for chrome and run to see if it works

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
driver.get('https://uk.indeed.com/jobs?q=data+scientist&l=London,+Greater+London&start=0')
print("Page Title is : %s" %driver.title)

<ipython-input-18-c2042bbce4a8>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
<ipython-input-18-c2042bbce4a8>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')


Page Title is : Data Scientist Jobs in London - 2021 | Indeed.com


In [19]:
# Extract job cards info by finding elements with xpath

job_cards=driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')

<ipython-input-19-24cf765529a0>:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  job_cards=driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')


In [20]:
len(job_cards)

15

# Prototype the model with a single record 

In [170]:
job_card=job_cards[1]

In [171]:
# Title

title=job_card.find_element_by_xpath('.//h2[contains(@class,"jobTitle")]').text.strip('new').strip()

In [172]:
# Company name
# Using try-except method as there are some errors with company name, salary and description

try:
    company=job_card.find_element_by_xpath('.//span[@class="companyName"]').text
except AttributeError:
    company='None'

In [173]:
# Location

location = job_card.find_element_by_xpath('.//div[contains(@class,"companyLocation")]').text

In [174]:
# Salary

try:
    salary= job_card.find_element_by_xpath('.//div[@class="salary-snippet"]').text
except AttributeError:
    salary='Negotiable'

In [175]:
# Description

try:
    description = job_card.find_element_by_tag_name('li').text
except AttributeError:
    description='None'

# Generalize the model with a function

In [21]:
# Create a function to extract job data from a single record

def get_record(job_card):
    title=job_card.find_element_by_xpath('.//h2[contains(@class,"jobTitle")]').text.strip('new').strip()
    try:
        company=job_card.find_element_by_xpath('.//span[@class="companyName"]').text
    except:
        company='None'
    try:
        location = job_card.find_element_by_xpath('.//div[contains(@class,"companyLocation")]').text
    except AttributeError:
        company='None'
    try:
        salary= job_card.find_element_by_xpath('.//div[@class="salary-snippet"]').text
    except:
        salary = 'Negotiable'
    try:
        description = job_card.find_element_by_tag_name('li').text
    except AttributeError:
        description='None'

    
    record=(title,company,location,salary,description)
    return record

In [177]:
# Run an example

records=[]

for job_card in job_cards:
    record=get_record(job_card)
    records.append(record)

In [178]:
records[0]

('Graduate Data Scientist - Modelling (Sep 2022)',
 'Funding Circle UK',
 'London EC4V',
 'Negotiable',
 'Using specific tools and languages such as R, Python, SQL and Tableau for reporting and analysis on our AWS data platform.')

# Put things together
# Record data until the end of the job search result and return the record:

In [ ]:
def get_url(position,location):
    template="https://uk.indeed.com/jobs?q={}&l={}"
    url=template.format(position,location)
    return url

def get_record(job_card):
    title=job_card.find_element_by_xpath('.//h2[contains(@class,"jobTitle")]').text.strip('new').strip()
    try:
        company=job_card.find_element_by_xpath('.//span[@class="companyName"]').text
    except:
        company='None'
    try:
        location = job_card.find_element_by_xpath('.//div[contains(@class,"companyLocation")]').text
    except AttributeError:
        company='None'
    try:
        salary= job_card.find_element_by_xpath('.//div[@class="salary-snippet"]').text
    except:
        salary = 'Negotiable'
    try:
        description = job_card.find_element_by_tag_name('li').text
    except AttributeError:
        description='None'    
    record=(title,company,location,salary,description)
    return record

# The main program

def main(position, location):
    records = []
    url = get_url(position, location)
    
    while True:
        try:
            url = driver.find_element_by_xpath('.//a[contains(@aria-label,"Next")]').get_attribute('href')
        except AttributeError:
            break

        driver = webdriver.Chrome(chrome_options = options, executable_path = r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
        driver.get(url)
        job_cards = driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')
        driver.implicitly_wait(3) 

        # record the data:
        for job_card in job_cards:
            record = get_record(job_card)
            records.append(record)

        # save the recorded data:
    with open('job_data.csv','w',newline = '',encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title','Company','Location','Salary','Description'])
        writer.writerows(records)

# Demo with 5 Urls

In [22]:
# Run the program with 5 pieces of url

def main(position, location):
    records = []
    url = get_url(position, location)

    for number in range(0,5):
        url='https://uk.indeed.com/jobs?q=data+scientist&l=London,+Greater+London&start={}'.format(number)

        driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
        driver.get(url)
        job_cards=driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')
        driver.implicitly_wait(3) 

        # record the data:
        for job_card in job_cards:
            record=get_record(job_card)
            records.append(record)

        # save the recorded data:
    with open('job_data.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title','Company','Location','Salary','Description'])
        writer.writerows(records)

In [23]:
# run the main function:
main('data scientist', 'London')

<ipython-input-22-1eebe8f9cedb>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
<ipython-input-22-1eebe8f9cedb>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=r'C:\Users\khoip\AppData\Local\Google\Chrome\User Data\Default\chromedriver.exe')
<ipython-input-22-1eebe8f9cedb>:10: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  job_cards=driver.find_elements_by_xpath('//div[contains(@class,"job_seen_beacon")]')
C:\Users\khoip\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:393: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element